In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.io import mmwrite
import h5py
import loompy
import os, re

In [2]:
path = '/lustre/orion/syb111/proj-shared/Projects/scrna-seq/data/human/heart/healthy/sanger/raw/hca_heart_global_ctl200723_freeze.h5ad'
path = '/lustre/orion/syb111/proj-shared/Projects/scrna-seq/data/human/GTEx_8_tissues_snRNAseq_atlas_071421.public_obs.h5ad'
path = '/gpfs/alpine2/syb112/proj-shared/Projects/scrna-seq/data/human/pbmc/healthy/washington/raw/all_pbmcs/all_pbmcs_rna.h5ad'
path = '/gpfs/alpine2/syb112/proj-shared/Projects/scrna-seq/data/human/pbmc/healthy/washington/raw/all_pbmcs/all_pbmcs_rna_harmony.h5ad'

# Loompy

In [1]:
import loompy

/lustre/orion/syb111/proj-shared/Personal/jmerlet/envs/conda/frontier/cytograph/lib/python3.9/site-packages/numba/np/ufunc/parallel.py:365: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 10002. The TBB threading layer is disabled.
  warnings.warn(problem)


In [13]:
import numpy as np
import scipy.sparse as sparse
import os

In [3]:
path = '/lustre/orion/syb111/proj-shared/Projects/scrna-seq/data/human/brain/healthy/nemo/raw/linnarson/adult_human_20221007.loom'

In [4]:
ds = loompy.connect(path)
ds

,,,Age,42.0,42.0,29.0,29.0,50.0,29.0,29.0,29.0,29.0,29.0,...
,,,CellCycle,0.0010152284263959391,0.0005431830526887561,0.0028076743097800658,0.0005899705014749262,0.0007797270955165692,0.0013729977116704807,0.0007272727272727272,0.001171303074670571,0.0019782393669634025,0.0027303754266211604,...
,,,CellID,10X376_7:GAACGTTGTATCGCTA,10X376_8:GAATCGTTCGATACGT,10X393_5:GCGTTTCCATAGATGA,10X377_4:CTCAACCTCATTTCCA,10X220_8:CGGGACTTCGGTGAAG,10X230_5:GTGTAACGTACCTTCC,10X264_3:CAGTGCGAGCCTTCTC,10X418_1:TGGATGTGTATCGATC,10X393_4:TGAGGGAGTGTCATTG,10X319_5:CTCCCAATCCAACACA,...
,,,Chemistry,v3,v3,v3,v3,v3,v3,v3,v3.1,v3,v3,...
,,,Clusters,0,0,0,0,0,0,0,0,0,0,...
,,,Donor,H19.30.001,H19.30.001,H19.30.002,H19.30.002,H18.30.002,H19.30.002,H19.30.002,H19.30.002,H19.30.002,H19.30.002,...
,,,NGenes,1254.0,1038.0,1227.0,1043.0,1394.0,1250.0,953.0,1527.0,718.0,1582.0,...
,,,ROIGroupCoarse,Cerebral cortex,Cerebral cortex,Cerebellum,Thalamus,Cerebral nuclei,Hippocampus,Cerebral cortex,Pons,Hippocampus,Cerebral nuclei,...
,,,ROIGroupFine,CerebralCortex,CerebralCortex,Cerebellum,Thalamus,BasalForebrain,Hippocampus,CerebralCortex,Pons,Hippocampus,BasalForebrain,...
,,,Roi,ITG,ITG,CbDN,LP,Cla,DGR-CA4Rpy,LEC,PB,CA4C-DGC,GPe,...
,,,SampleID,10X376_7,10X376_8,10X393_5,10X377_4,10X220_8,10X230_5,10X264_3,10X418_1,10X393_4,10X319_5,...


In [5]:
ds.close()

In [6]:
with loompy.connect(path) as ds:
    coarse = ds.ca['ROIGroupCoarse']
    hypo_idx = coarse == 'Hypothalamus'
    all_bcs = ds.ca['CellID']
    hypo_bcs = all_bcs[hypo_idx]

In [7]:
print(f'# bcs: {len(all_bcs)}')
print(f'# hypo bcs: {len(hypo_bcs)}')

# bcs: 3369219
# hypo bcs: 134471


In [8]:
hypo_sample_ids = [bc[:8] for bc in hypo_bcs]
uniq_hypo_sample_ids, hypo_sample_counts = np.unique(hypo_sample_ids, return_counts=True)

### For the Root.yaml file

In [9]:
uniq_hypo_sample_ids

array(['10X190_5', '10X190_6', '10X192_7', '10X192_8', '10X193_3',
       '10X193_4', '10X203_7', '10X203_8', '10X360_7', '10X360_8',
       '10X362_3', '10X362_4', '10X362_5', '10X362_6', '10X376_5',
       '10X376_6', '10X380_5', '10X380_6', '10X380_7', '10X380_8',
       '10X389_1', '10X389_2', '10X392_3', '10X392_4'], dtype='<U8')

In [10]:
hypo_sample_counts

array([6326, 6847, 6380, 6177, 4983, 5381, 7519, 7460, 9757, 8971, 5847,
       5367, 3319, 3258, 4148, 4397, 5436, 6046, 3475, 3790, 5119, 4980,
       4389, 5099])

### Writing each Hypo sample to a .loom file

In [11]:
hypo_bcs_by_sample = {}
for hypo_sample_id in uniq_hypo_sample_ids:
    hypo_sample_bcs = np.array([True if bc[:8] == hypo_sample_id else False for bc in all_bcs])
    hypo_bcs_by_sample[hypo_sample_id] = hypo_sample_bcs

In [25]:
with loompy.connect(path) as ds:
    ra = {}
    for key, value in ds.ra.items():
        ra[key] = value

In [ ]:
hypo_loom_dir = '/lustre/orion/syb111/proj-shared/Projects/scrna-seq/data/human/brain/healthy/nemo/raw/linnarson/hypo_looms'
with loompy.connect(path) as ds:
    for sample, idx in hypo_bcs_by_sample.items():
        print(sample)
        out_path = os.path.join(hypo_loom_dir, str(sample) + '.loom')
        ca = {}
        for key, value in ds.ca.items():
            ca[key] = value[idx]
        mtx = ds[:, idx]
        loompy.create(out_path, mtx, ra, ca)
    # kernel dies on below
    #hypo_ds = ds[:, hypo_idx].sparse()
    

10X190_5
10X190_6
10X192_7
10X192_8
10X193_3
10X193_4
10X203_7


# H5

In [3]:
# to see all the hd5 group names
with h5py.File(path, 'r') as f:
    for key in f.keys():
        #if key == 'obs':
            #bc_meta = list(f[key])
        print(f"{key}: {list(f[key])}")

attrs: ['CreationDate', 'LOOM_SPEC_VERSION']
col_attrs: ['Age', 'CellCycle', 'CellID', 'Chemistry', 'Clusters', 'Donor', 'NGenes', 'ROIGroupCoarse', 'ROIGroupFine', 'Roi', 'SampleID', 'Sex', 'Subclusters', 'Tissue', 'TotalUMI']
col_graphs: []
layers: []


KeyboardInterrupt: 

In [6]:
with h5py.File(path, 'r') as f:
    bcs = f['obs']['_index'][()]
    genes = f['var']['_index'][()]

In [7]:
genes[:10]

array(['MIR1302-2HG', 'FAM138A', 'OR4F5', 'AL627309.1', 'AL627309.3',
       'AL627309.2', 'AL627309.5', 'AL627309.4', 'AP006222.2',
       'AL732372.1'], dtype=object)

In [8]:
bcs[:10]

array(['ALAW-AS044-1_AAAGCAAGTAGCTTGT-1',
       'ALAW-AS044-1_AAAGCAATCTCAAGTG-1',
       'ALAW-AS044-1_AAAGTAGGTGTTGAGG-1',
       'ALAW-AS044-1_AACCATGCACCGATAT-1',
       'ALAW-AS044-1_AACCATGCATGCCACG-1',
       'ALAW-AS044-1_AACTCAGTCGCCTGTT-1',
       'ALAW-AS044-1_AACTCCCGTCTCCATC-1',
       'ALAW-AS044-1_AACTCCCTCATAACCG-1',
       'ALAW-AS044-1_AACTGGTAGAGCCCAA-1',
       'ALAW-AS044-1_AAGCCGCAGTGCCAGA-1'], dtype=object)

In [14]:
with h5py.File(path, 'r') as f:
    for key in f['var']['__categories'].keys():
        print(f"{key}: {f['var']['__categories'][key][()]}")

Alias symbols: ['1-8D, DSPA2c' '1-8U, DSPA2b' '1C7, NKp30, CD337' ...
 'y+LAT-2, KIAA0245, LAT3, LAT-2' 'yt14r, CHR14SYT, Strep14' 'α4']
Approved name: ['1,4-alpha-glucan branching enzyme 1'
 '1-acylglycerol-3-phosphate O-acyltransferase 1'
 '1-acylglycerol-3-phosphate O-acyltransferase 2' ...
 'zymogen granule protein 16' 'zymogen granule protein 16B' 'zyxin']
Approved symbol: ['A1BG' 'A1CF' 'A2M' ... 'ZZEF1' 'ZZZ3' 'nan']
Chromosome: ['1' '10' '11' '12' '13' '14' '15' '16' '17' '18' '19' '2' '20' '21' '22'
 '3' '4' '5' '6' '7' '8' '9' 'X' 'Y']
Previous symbols: ['A1S9T, GXP1, UBE1' 'A2LD1' 'A2MRAP, RAP' ... 'ZYG11' 'ZZANK1' 'nan']
Source: ['ensembl' 'ensembl_havana' 'havana']
Status: ['Approved' 'nan']
Strand: ['+' '-']
gene_biotype: ['protein_coding']
gene_name: ['A1BG' 'A1CF' 'A2M' ... 'ZYX' 'ZZEF1' 'ZZZ3']
gene_source: ['ensembl' 'ensembl_havana' 'havana']


In [15]:
# printing head of each var
with h5py.File(path, 'r') as f:
    for key in f.keys():
        if key == 'var':
            for var_key in f[key].keys():
                print(f'{var_key}: {f[key][var_key]}')

Alias symbols: <HDF5 dataset "Alias symbols": shape (17695,), type "<i2">
Approved name: <HDF5 dataset "Approved name": shape (17695,), type "<i2">
Approved symbol: <HDF5 dataset "Approved symbol": shape (17695,), type "<i2">
Chromosome: <HDF5 dataset "Chromosome": shape (17695,), type "|i1">
End: <HDF5 dataset "End": shape (17695,), type "<i4">
Previous symbols: <HDF5 dataset "Previous symbols": shape (17695,), type "<i2">
Source: <HDF5 dataset "Source": shape (17695,), type "|i1">
Start: <HDF5 dataset "Start": shape (17695,), type "<i4">
Status: <HDF5 dataset "Status": shape (17695,), type "|i1">
Strand: <HDF5 dataset "Strand": shape (17695,), type "|i1">
__categories: <HDF5 group "/var/__categories" (11 members)>
_index: <HDF5 dataset "_index": shape (17695,), type "|O">
gene_biotype: <HDF5 dataset "gene_biotype": shape (17695,), type "|i1">
gene_coding_length: <HDF5 dataset "gene_coding_length": shape (17695,), type "<i4">
gene_ids: <HDF5 dataset "gene_ids": shape (17695,), type "|

In [18]:
with h5py.File(path, 'r') as f:
    print(f"{key}: {f['var']['gene_source'][()]}")

varm: [0 1 1 ... 1 1 1]


In [59]:
with h5py.File(path, 'r') as hf:
    for name in ['age_group', 'cell_type', 'cell_states', 'region', 'source']:
        data = hf['obs'][name]
        
        print(f'({len(np.unique(data))}) {name}: {hf["obs"][name][()]}')

(7) age_group: [2 2 2 ... 4 4 4]
(13) cell_type: [10 11 11 ...  2  2  2]
(67) cell_states: [52 62 62 ... 12 16 15]
(6) region: [0 0 0 ... 0 0 0]
(3) source: [2 2 2 ... 0 0 0]


In [ ]:
{'age_group': {0: '40-45', 1: '45-50'}, 'cell_type': {}}

In [77]:
#names = {'age_group': 2, 'cell_type': 5, 'cell_states': 4, 'region': 6, 'source': 10}

with h5py.File(path, 'r') as hf:
    x = hf['obs']['__categories']
    converter = {}
    for i, category in enumerate(x):
        if i in [2, 4, 5, 8, 10]:
            converter[category] = {}
            for j, txt in enumerate(x[category][()]):
                converter[category][j] = txt
            #converter[i] = str(x[category][()])
    for i, thing in enumerate(x):
        print(f'{i}: {x[thing][()]}')

0: ['No' 'Yes']
1: ['No' 'Yes']
2: ['40-45' '45-50' '50-55' '55-60' '60-65' '65-70' '70-75']
3: ['Harvard-Nuclei' 'Sanger-CD45' 'Sanger-Cells' 'Sanger-Nuclei']
4: ['Adip1' 'Adip2' 'Adip3' 'Adip4' 'B_cells' 'CD4+T_cytox' 'CD4+T_tem'
 'CD8+T_cytox' 'CD8+T_tem' 'CD16+Mo' 'DOCK4+MØ1' 'DOCK4+MØ2' 'EC1_cap'
 'EC2_cap' 'EC3_cap' 'EC4_immune' 'EC5_art' 'EC6_ven' 'EC7_atria' 'EC8_ln'
 'EC9_FB-like' 'EC10_CMC-like' 'FB1' 'FB2' 'FB3' 'FB4' 'FB5' 'FB6' 'FB7'
 'IL17RA+Mo' 'LYVE1+MØ1' 'LYVE1+MØ2' 'LYVE1+MØ3' 'Mast' 'Meso' 'CD14+Mo'
 'Mo_pi' 'MØ_AgP' 'MØ_mod' 'NC1' 'NC2' 'NC3' 'NC4' 'NC5' 'NC6' 'NK' 'NKT'
 'NØ' 'PC1_vent' 'PC2_atria' 'PC3_str' 'PC4_CMC-like' 'SMC1_basic'
 'SMC2_art' 'aCM1' 'aCM2' 'aCM3' 'aCM4' 'aCM5' 'doublets' 'DC' 'nan'
 'vCM1' 'vCM2' 'vCM3' 'vCM4' 'vCM5']
5: ['Adipocytes' 'Atrial_Cardiomyocyte' 'Endothelial' 'Fibroblast' 'Lymphoid'
 'Mesothelial' 'Myeloid' 'Neuronal' 'NotAssigned' 'Pericytes'
 'Smooth_muscle_cells' 'Ventricular_Cardiomyocyte' 'doublets']
6: ['D1' 'D2' 'D3' 'D4' 'D

In [78]:
converter

{'age_group': {0: '40-45',
  1: '45-50',
  2: '50-55',
  3: '55-60',
  4: '60-65',
  5: '65-70',
  6: '70-75'},
 'cell_states': {0: 'Adip1',
  1: 'Adip2',
  2: 'Adip3',
  3: 'Adip4',
  4: 'B_cells',
  5: 'CD4+T_cytox',
  6: 'CD4+T_tem',
  7: 'CD8+T_cytox',
  8: 'CD8+T_tem',
  9: 'CD16+Mo',
  10: 'DOCK4+MØ1',
  11: 'DOCK4+MØ2',
  12: 'EC1_cap',
  13: 'EC2_cap',
  14: 'EC3_cap',
  15: 'EC4_immune',
  16: 'EC5_art',
  17: 'EC6_ven',
  18: 'EC7_atria',
  19: 'EC8_ln',
  20: 'EC9_FB-like',
  21: 'EC10_CMC-like',
  22: 'FB1',
  23: 'FB2',
  24: 'FB3',
  25: 'FB4',
  26: 'FB5',
  27: 'FB6',
  28: 'FB7',
  29: 'IL17RA+Mo',
  30: 'LYVE1+MØ1',
  31: 'LYVE1+MØ2',
  32: 'LYVE1+MØ3',
  33: 'Mast',
  34: 'Meso',
  35: 'CD14+Mo',
  36: 'Mo_pi',
  37: 'MØ_AgP',
  38: 'MØ_mod',
  39: 'NC1',
  40: 'NC2',
  41: 'NC3',
  42: 'NC4',
  43: 'NC5',
  44: 'NC6',
  45: 'NK',
  46: 'NKT',
  47: 'NØ',
  48: 'PC1_vent',
  49: 'PC2_atria',
  50: 'PC3_str',
  51: 'PC4_CMC-like',
  52: 'SMC1_basic',
  53: 'SMC2_art',

In [ ]:
# age_group: 2
# cell_type: 5
# cell_state: 4
# percent_mito:

In [79]:
#mapper = {'age_group': 2, 'cell_type': 5, 'cell_states': 4, 'region': 6, 'source': 10}

with h5py.File(path, 'r') as f:
    # gene x cell matrix
    data = f['X']['data'][()]
    indices = f['X']['indices'][()]
    indptr = f['X']['indptr'][()]
    sparse_mtx = csr_matrix((data, indices, indptr)).transpose()
    # bc and gene ids
    bc_ids = f['obs']['_index'][()]
    hgnc_gene_ids = f['var']['_index'][()]
    #ens_gene_ids = f['var']['gene_ids'][()]
    # cell types
    bc_meta_to_keep = ['age_group', 'cell_type', 'cell_states', 'region', 'source']

    bc_metadata_to_keep = []
    for meta in bc_meta_to_keep:
        metadata = f['obs'][meta][()]
        my_dict = converter[meta]
        my_dict = pd.Series(my_dict)
        metadata = my_dict[metadata].to_numpy().T
        bc_metadata_to_keep.append(metadata)

In [84]:
# how many bcs?
print(len(bc_ids))

486134


In [82]:
# write the matrix to file in Market Exchange Format (10X's .mtx format) and use 10X filename
#out_mtx_path = '/gpfs/alpine/syb105/proj-shared/Personal/jmerlet/projects/cell_atlasses/data/human/lister_lab/matrix.mtx'
out_mtx_path = '/lustre/orion/syb111/proj-shared/Projects/scrna-seq/data/human/heart/healthy/sanger/raw/matrix.mtx'
mmwrite(out_mtx_path, sparse_mtx, field='integer')

In [85]:
# write the bcs and bc cell type metadata to file
# match 10x filename and formatting for Seurat read-in
out_bc_path = '/lustre/orion/syb111/proj-shared/Projects/scrna-seq/data/human/heart/healthy/sanger/raw/barcodes.tsv'
out_bc_data = pd.DataFrame(bc_ids.astype(str))
out_bc_data.to_csv(out_bc_path, sep='\t', index=False, header=False)
# filename doesn't matter for metadata
out_bc_meta_path = '/lustre/orion/syb111/proj-shared/Projects/scrna-seq/data/human/heart/healthy/sanger/meta/bc_meta.tsv'
out_bc_metadata = pd.DataFrame(data=bc_metadata_to_keep).T
out_bc_metadata.index = bc_ids.astype(str)
out_bc_metadata.columns = bc_meta_to_keep
out_bc_metadata.to_csv(out_bc_meta_path, sep='\t')

In [87]:
# write the HGNC symbols to file using 10X filename and formatting
out_gene_path = '/gpfs/alpine/syb105/proj-shared/Personal/jmerlet/projects/cell_atlasses/data/human/lister_lab/features.tsv'
out_gene_path = '/lustre/orion/syb111/proj-shared/Projects/scrna-seq/data/human/heart/healthy/sanger/raw/features.tsv'
str_fill_col = np.repeat('Gene', len(hgnc_gene_ids))
out_gene_data = pd.DataFrame(data=[hgnc_gene_ids.astype(str), hgnc_gene_ids.astype(str), str_fill_col]).T
out_gene_data.to_csv(out_gene_path, sep='\t', index=False, header=False)